<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/mt5-base_5000ds_1e-4lr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Sun Jan 29 10:07:39 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   58C    P0    27W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install simpletransformers
!pip install torch
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# # build dataset

# import pandas as pd

# data = pd.read_csv('/content/gdrive/MyDrive/dataset5000pair.csv', usecols=['Thai', 'English'])

# !rm -rf '/content/train.csv'
# !rm -rf '/content/eval.csv'
# !rm -rf '/content/test.csv'

# from sklearn.model_selection import train_test_split

# train_data, eval_test = train_test_split(data, train_size=0.8, shuffle=True, random_state=1234)
# eval_data, test_data = train_test_split(eval_test, train_size=0.5, shuffle=True, random_state=1234)
# train_data.to_csv('/content/train.csv', index=False)
# eval_data.to_csv('/content/eval.csv', index=False)
# test_data.to_csv('/content/test.csv', index=False)

# train_data = pd.read_csv('/content/train.csv')
# eval_data = pd.read_csv('/content/eval.csv')
# test_data = pd.read_csv('/content/test.csv')

# !rm -rf '/content/train.csv'
# !rm -rf '/content/eval.csv'
# !rm -rf '/content/test.csv'

# import csv

# with open('/content/train.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['prefix', 'input_text', 'target_text'])
#   for i in range(len(train_data)):
#     writer.writerow(['translate thai to english', train_data['Thai'][i], train_data['English'][i]])
#     writer.writerow(['translate english to thai', train_data['English'][i], train_data['Thai'][i]])
# with open('/content/eval.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['prefix', 'input_text', 'target_text'])
#   for i in range(len(eval_data)):
#     writer.writerow(['translate thai to english', eval_data['Thai'][i], eval_data['English'][i]])
#     writer.writerow(['translate english to thai', eval_data['English'][i], eval_data['Thai'][i]])
# with open('/content/test.csv', 'w') as csv_file:
#   writer = csv.writer(csv_file)
#   writer.writerow(['prefix', 'input_text', 'target_text'])
#   for i in range(len(test_data)):
#     writer.writerow(['translate thai to english', test_data['Thai'][i], test_data['English'][i]])
#     writer.writerow(['translate english to thai', test_data['English'][i], test_data['Thai'][i]])

# pd.read_csv('/content/train.csv')

In [5]:
import pandas as pd

train_data = pd.read_csv('/content/gdrive/MyDrive/train.csv')
train_data

,prefix,input_text,target_text
0,translate thai to english,การวิจัยมีวัตถุประสงค์เพื่อ พัฒนาระบบสารสนเทศ...,The objectives of the research were to develo...
1,translate english to thai,The objectives of the research were to develo...,การวิจัยมีวัตถุประสงค์เพื่อ พัฒนาระบบสารสนเทศ...
2,translate thai to english,การวิจัยเริ่มจากการวิเคราะห์ปัญหาของผังโรงงานใ...,The research begins with an analysis of the pr...
3,translate english to thai,The research begins with an analysis of the pr...,การวิจัยเริ่มจากการวิเคราะห์ปัญหาของผังโรงงานใ...
4,translate thai to english,อัลกอริทึมตามปัจจุบันใช้การเปลี่ยนแปลงของกระแส...,The current based algorithm directly uses the ...
...,...,...,...
7995,translate english to thai,The objective of this study is to select appro...,การศึกษาในครั้งนี้ มีวัตถุประสงค์เพื่อคัดเลือก...
7996,translate thai to english,ผลการทดลองพบว่าการผลิตไบโอดีเซลจากน้ำมันปาล์มบ...,The experimental results found that biodiesel ...
7997,translate english to thai,The experimental results found that biodiesel ...,ผลการทดลองพบว่าการผลิตไบโอดีเซลจากน้ำมันปาล์มบ...
7998,translate thai to english,ผลการวิจัยพบว่าฝาครอบเครื่องมือถูลดข้อบกพร่องเ...,The results of the research showed that the Ru...


In [6]:
eval_data = pd.read_csv('/content/gdrive/MyDrive/eval.csv')
eval_data

,prefix,input_text,target_text
0,translate thai to english,สรุปได้ว่าภาพจำลองหลอนนั้นคล้ายกับภาพเอกซเรย์ค...,"In conclusion, the phantom simulation images w..."
1,translate english to thai,"In conclusion, the phantom simulation images w...",สรุปได้ว่าภาพจำลองหลอนนั้นคล้ายกับภาพเอกซเรย์ค...
2,translate thai to english,ปูแสม (วรุณยุ้ย) เป็นปูทะเลเศรษฐกิจชนิดหนึ่งขอ...,Sundaic paddler crab (Varuna yui) is one of ec...
3,translate english to thai,Sundaic paddler crab (Varuna yui) is one of ec...,ปูแสม (วรุณยุ้ย) เป็นปูทะเลเศรษฐกิจชนิดหนึ่งขอ...
4,translate thai to english,ด้วยเหตุนี้ ระบบของเราจึงได้รับประสิทธิภาพการท...,"Accordingly, our system achieves a better dryi..."
...,...,...,...
995,translate english to thai,This condition can be control amount of planti...,เงื่อนไขนี้สามารถควบคุมปริมาณการป้อนวัสดุปลูกต...
996,translate thai to english,ข้อมูลเริ่มรวบรวมจากวัตถุดิบและขั้นตอนบรรจุภัณ...,The data was beginning to collect from raw mat...
997,translate english to thai,The data was beginning to collect from raw mat...,ข้อมูลเริ่มรวบรวมจากวัตถุดิบและขั้นตอนบรรจุภัณ...
998,translate thai to english,งานวิจัยนี้มีวัตถุประสงค์เพื่อพัฒนาสื่อโมชันกร...,This research aims to develop motion graphics ...


In [7]:
test_data = pd.read_csv('/content/gdrive/MyDrive/test.csv')
test_data

,prefix,input_text,target_text
0,translate thai to english,ผลการวิจัยพบว่า แบบทดสอบวินิจฉัยข้อบกพร่องราย...,The research found that diagnostic tests for ...
1,translate english to thai,The research found that diagnostic tests for ...,ผลการวิจัยพบว่า แบบทดสอบวินิจฉัยข้อบกพร่องราย...
2,translate thai to english,ผลการวิจัยพบว่าความหนาของแผ่นเหล็กมีผลต่อความถ...,The result found that the thickness of steel p...
3,translate english to thai,The result found that the thickness of steel p...,ผลการวิจัยพบว่าความหนาของแผ่นเหล็กมีผลต่อความถ...
4,translate thai to english,จากกระบวนการตรวจสอบพบว่าเทคนิคการระบายความร้อน...,"From the review process, it is found that the ..."
...,...,...,...
995,translate english to thai,In accordance with the standard of herbal comp...,ตามมาตรฐานผลิตภัณฑ์ลูกประคบสมุนไพร. ผลการประเม...
996,translate thai to english,โดยที่องค์ประกอบของระบบมีความเหมาะสมในระดับสูงสุด,Where the system components were suitable at t...
997,translate english to thai,Where the system components were suitable at t...,โดยที่องค์ประกอบของระบบมีความเหมาะสมในระดับสูงสุด
998,translate thai to english,ในงานวิจัยนี้มีวัตถุประสงค์ที่จะนำเสนอวิธีการป...,"In this research, the objective is to present ..."


In [8]:
# config model

import logging
import torch
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transf-ormers")
transformers_logger.setLevel(logging.WARNING)

train_df = pd.read_csv('/content/gdrive/MyDrive/train.csv').astype(str)
eval_df = pd.read_csv('/content/gdrive/MyDrive/eval.csv').astype(str)

train_df["prefix"] = ""
eval_df["prefix"] = ""

model_args = T5Args()
model_args.train_batch_size = 8
model_args.eval_batch_size = 8
model_args.num_train_epochs = 8
model_args.learning_rate = 1e-4
model_args.evaluate_during_training = True
model_args.evaluate_during_training_steps = 30000
model_args.use_multiprocessing = False
model_args.fp16 = False
model_args.save_steps = -1
model_args.save_eval_checkpoints = False
model_args.no_cache = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.preprocess_inputs = False
model_args.num_return_sequences = 1
model_args.wandb_project = "MT5 Thai-English Translation"

device = torch.cuda.is_available()
model = T5Model("mt5", "google/mt5-base", args=model_args, use_cuda=device)

In [9]:
# !wandb login --relogin

In [10]:
!rm -rf '/content/outputs'
!rm -rf '/content/runs'
!rm -rf '/content/wandb'

# Train the model
model.train_model(train_df, eval_data=eval_df)

  0%|          | 0/8000 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3707: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprec

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: defymiygfl (sigmarai). Use `wandb login --relogin` to force relogin


Running Epoch 0 of 8:   0%|          | 0/1000 [00:00<?, ?it/s]

...

Running Epoch 6 of 8:   0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

Running Epoch 7 of 8:   0%|          | 0/1000 [00:00<?, ?it/s]

  0%|          | 0/1000 [00:00<?, ?it/s]

(8000,
 {'global_step': [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000],
  'eval_loss': [3.4533920269012452,
   2.5916166610717775,
   2.1688839502334596,
   1.9479573669433594,
   1.8392701215744018,
   1.7155731482505798,
   1.6720699672698975,
   1.593535261631012],
  'train_loss': [4.639876365661621,
   3.5887465476989746,
   3.0157856941223145,
   2.656953811645508,
   2.4283852577209473,
   2.083871841430664,
   1.7061355113983154,
   2.311457633972168]})

In [11]:
# Evaluate the model
result = model.eval_model(eval_df)

  0%|          | 0/1000 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/125 [00:00<?, ?it/s]

In [12]:
import logging
import sacrebleu
import pandas as pd
from simpletransformers.t5 import T5Model, T5Args

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 96
model_args.length_penalty = 1
model_args.num_beams = 10

model = T5Model("mt5", "/content/outputs", args=model_args)

In [13]:
valid = pd.read_csv("/content/gdrive/MyDrive/eval.csv").astype(str)

thai_truth = [valid.loc[valid["prefix"] == "translate english to thai"]["target_text"].tolist()]
to_thai = valid.loc[valid["prefix"] == "translate english to thai"]["input_text"].tolist()

english_truth = [valid.loc[valid["prefix"] == "translate thai to english"]["target_text"].tolist()]
to_english = valid.loc[valid["prefix"] == "translate thai to english"]["input_text"].tolist()

In [14]:
english_preds = model.predict(to_english)
then_bleu = sacrebleu.corpus_bleu(english_preds, english_truth)
print("Thai to English: ", then_bleu.score)

thai_preds = model.predict(to_thai)
enth_bleu = sacrebleu.corpus_bleu(thai_preds, thai_truth)
print("English to Thai: ", enth_bleu.score)

Generating outputs:   0%|          | 0/63 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/500 [00:00<?, ?it/s]

Thai to English:  23.930544579970658


Generating outputs:   0%|          | 0/63 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/500 [00:00<?, ?it/s]

English to Thai:  21.470776036234508


In [15]:
from simpletransformers.t5 import T5Model

model = T5Model("mt5", "/content/outputs")
to_predict = ['โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ',
              'และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก',
              'โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน',
              'ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน',
              'และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน',
              'แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)',
              'ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน',
              'และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์',
              'ในขั้นตอนการเลือกแบบจําลองการเรียนรู้เชิงลึกเพื่อนําไปประมวลผลบนเครื่อง Jetson Nano']
# to_predict = to_predict[0].split()
predictions = model.predict(to_predict)
for i,info in enumerate(predictions):
  print(to_predict[i] + '\n' + info + '\n')

Generating outputs:   0%|          | 0/2 [00:00<?, ?it/s]

Decoding outputs:   0%|          | 0/9 [00:00<?, ?it/s]

โครงงานวิศวกรรมนี้เป็นการพัฒนาระบบตรวจจับ
The objective of this engineering project is to develop a tracking system.

และจําแนกโรคพืชจากภาพถ่ายใบพืชเมล่อนด้วยการเรียนรู้เชิงลึก
The species of plant diseases were classified from the soil images of melon leave

โดยใช้เครื่อง Jetson Nano ในการประมวลผลภาพจากกล้องในโรงเรือน
The jetson NANO machine was used to processing image from the dashboard and houses

ซึ่งในการพัฒนาแบบจําลองการเรียนรู้เชิงลึกมีการเก็บรวบรวมภาพของใบเมล่อนจากโรงเรือน
The objective of the deep learning model was collected the image of a melon from the

และจัดทําสัญลักษณ์ความผิดปกติของใบเมล่อน
The mutation of the melon leaves was developed.

แล้วนําชุดข้อมูลที่ได้ไปใช้ในการฝึกแบบจําลองตรวจจับวัตถุในภาพ (Object Detector)
The data were then applied to simulate the object detector simulation.

ซึ่งจะสามารถตรวจพบความผิดปกติของใบเมล่อน
The findings can be observed in the melon leaves.

และเมื่อระบบตรวจพบความผิดปกติจะแจ้งเตือนไปยังผู้ดูแลผ่านแอปพลิเคชันไลน์
When the system detec